In [1]:
import pyarrow
import pymapd
import pandas as pd
import geopandas as gpd 
from shapely.geometry import Point

In [2]:
def cursor2df(cursor):
    col_names = [c.name for c in cursor.description]
    return pd.DataFrame(cursor.fetchall(), columns=col_names)

In [3]:
con = pymapd.connect(host='localhost', user='admin', password='HyperInteractive')

## Array field

In [4]:
con.execute('''
DROP TABLE IF EXISTS mytablearray;
''')

con.execute('''
CREATE TABLE mytablearray (
    col_a INT[],
    col_b INT,
    col_c text,
    col_d double
);
''')

In [5]:
# df = pd.DataFrame({'a': pd.array([pd.array([1, 2], dtype='int'), pd.array([4, 5, 6, 7], dtype='int')])})
df = pd.DataFrame({
    'a': [[1, 2], [4, 5, 6, 7]],
    'b': [1, 2],
    'c': ['a', 'b'],
    'd': [1.1, 2.2]
})
df

,a,b,c,d
0,"[1, 2]",1,a,1.1
1,"[4, 5, 6, 7]",2,b,2.2


In [6]:
df.dtypes

a     object
b      int64
c     object
d    float64
dtype: object

In [7]:
se = pd.Series([[1., 2.], [4., 5., 6., 7.]])
se.apply(lambda l: [int(v) for v in l])

0          [1, 2]
1    [4, 5, 6, 7]
dtype: object

In [8]:
con.load_table('mytablearray', df, method='columnar')

/home/xmn/dev/quansight/omnisci-project/pymapd/pymapd/_pandas_loaders.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[c] = data[c].astype('int64')


In [9]:
cur = con.execute('select * from mytablearray')

In [10]:
df = cursor2df(cur)

In [11]:
df

,col_a,col_b,col_c,col_d
0,"[1, 2]",1,a,1.1
1,"[4, 5, 6, 7]",2,b,2.2


## Geospatial fields

In [12]:
table_name = 'mytablegeo'
con.execute('''
DROP TABLE IF EXISTS {};
'''.format(table_name))

con.execute('''
CREATE TABLE {} (
    col_a POINT,
    col_b INT,
    col_c text,
    col_d double
);
'''.format(table_name))

In [13]:
# df = pd.DataFrame({'a': pd.array([pd.array([1, 2], dtype='int'), pd.array([4, 5, 6, 7], dtype='int')])})
df = gpd.GeoDataFrame({
    'a': [Point(0, 0), Point(1, 1)],
    'b': [1, 2],
    'c': ['a', 'b'],
    'd': [1.1, 2.2]
})
df

,a,b,c,d
0,POINT (0 0),1,a,1.1
1,POINT (1 1),2,b,2.2


In [14]:
# note: not working
# con.load_table(table_name, df, method='columnar')

cur = con.execute('select * from {}'.format(table_name))

df = cursor2df(cur)

df

,col_a,col_b,col_c,col_d
